---
title: SpatioTemporal Field - T2M - Non-Stationary
subject: AEMET T2M
short_title: 4 - Analysis Region
authors:
  - name: J. Emmanuel Johnson
    affiliations:
      - CSIC
      - UCM
      - IGEO
    orcid: 0000-0002-6739-0053
    email: juanjohn@ucm.es
license: CC-BY-4.0
keywords: notation
---

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" # first gpu
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'FALSE'

import jax
jax.config.update('jax_platform_name', 'cpu')

import numpyro
import multiprocessing

num_devices = multiprocessing.cpu_count()
numpyro.set_platform("cpu")
numpyro.set_host_device_count(num_devices)

In [3]:
import autoroot
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
import pint_xarray
import arviz as az

from st_evt import CANDIDATE_STATIONS
from st_evt.utils import ExperimentPath
from st_evt.viz import (
    plot_scatter_ts,
    plot_histogram,
    plot_density,
    plot_return_level_gevd_manual_unc_multiple,
    plot_periods,
    plot_periods_diff,
    plot_spain,
    plot_density_multiple
)
from st_evt.extremes import estimate_return_level_gevd, calculate_exceedence_probs
from omegaconf import OmegaConf

import jax
import jax.random as jrandom
import jax.numpy as jnp
import pandas as pd

rng_key = jrandom.PRNGKey(123)

from numpyro.infer import Predictive
import arviz as az

import numpyro
from numpyro.diagnostics import hpdi
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS
import xarray as xr
import regionmask

import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FuncFormatter
from st_evt._src.modules.models.aemet import utils_region, utils_station
import seaborn as sns
from loguru import logger
sns.reset_defaults()
sns.set_context(context="talk", font_scale=0.7)


%config InlineBackend.figure_format = 'retina'
plt.style.use(
    "https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle"
)


%matplotlib inline
%load_ext autoreload
%autoreload 2

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


## Data

In [4]:
results_root_path = "/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten/results"
results_root_path = "/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results"
results_file_name = "az_nonstationary_gp_mcmc_redfeten.zarr"
results_data_path = Path(results_root_path).joinpath(results_file_name)

figures_path = Path(results_root_path).joinpath("figures/spain")

In [5]:
# results_root_path = "/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_lap_redfeten/results"
# results_file_name = "az_nonstationary_gp_lap_redfeten.zarr"
# results_data_path = Path(results_root_path).joinpath(results_file_name)

# figures_path = Path(results_root_path).joinpath("figures/spain")

In [6]:
# results_data_path = "/home/juanjohn/pool_data/dynev4eo/temp/results/scratch_pipelines/results/az_nonstationary_gp_lap_redfeten.zarr"
# figures_path = "./"#Path("/home/juanjohn/pool_data/dynev4eo/temp/scratch_pipelines/figures/nonstationary_gp_lap_redfeten/spain")

### MCMC Results

In [7]:
variable = "t2max"
covariate = "gmst"

In [8]:
az_ds = az.from_zarr(store=str(results_data_path))

## Model Errors - Posterior

### Negative Log-Likelihood

In [11]:
PLOT_VARIABLE = "nll"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [8]:

data_results = az.extract(az_ds, group="log_likelihood", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby(covariate)
idata = data_results[VARIABLE].rename(PLOT_VARIABLE).sum(dim=covariate)

In [9]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds_r",
    bounds = (-160, 10, -50)
)

2024-12-17 11:07:35.958 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:262 - Starting Static Figures...
2024-12-17 11:07:35.959 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:269 - Plotting NLL Density...
2024-12-17 11:07:36.449 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:282 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/nll/posterior/nll_density.pdf
2024-12-17 11:07:36.449 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:285 - Plotting Map...
2024-12-17 11:07:36.449 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:294 - Plotting Map of Spain...
2024-12-17 11:07:37.346 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:304 - Saved Figure: 
/ho

## Model Errors - Posterior Predictive

### I - Negative Log-Likelihood

In [10]:
PLOT_VARIABLE = "nll"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [11]:
az_ds = az.from_zarr(store=str(results_data_path))
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby(covariate)
idata = data_results["nll"].rename(PLOT_VARIABLE).sum(dim=covariate)

In [12]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable_redfeten(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds_r",
    bounds = (-160, 10, -50)
)

2024-12-17 11:08:04.108 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:321 - Starting Static Figures...
2024-12-17 11:08:04.109 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:328 - Plotting NLL Density...
2024-12-17 11:08:04.358 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:353 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/nll/posterior_predictive/nll_density_groups.pdf
2024-12-17 11:08:04.359 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:356 - Plotting Map...
2024-12-17 11:08:04.359 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:365 - Plotting Map of Spain...
2024-12-17 11:08:04.682 | DEBUG    | st_evt._src.modules.models.aemet.uti

### II - Residuals

In [13]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [14]:
PLOT_VARIABLE = "residuals"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [15]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
y_pred = data_results[variable].rename("y_pred")
y_true = data_results[f"{variable}_true"].rename("y_true")
idata = (y_true - y_pred).mean(dim=[covariate]).load().rename(PLOT_VARIABLE)

In [16]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable_redfeten(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds_r",
    bounds = (-7.5, 10, 7.5)
)

2024-12-17 11:08:18.341 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:321 - Starting Static Figures...
2024-12-17 11:08:18.342 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:328 - Plotting RESIDUALS Density...
2024-12-17 11:08:18.586 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:353 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/residuals/posterior_predictive/residuals_density_groups.pdf
2024-12-17 11:08:18.586 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:356 - Plotting Map...
2024-12-17 11:08:18.587 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:365 - Plotting Map of Spain...
2024-12-17 11:08:18.890 | DEBUG    | st_evt._src.module

### III - Absolute Residuals

In [17]:
PLOT_VARIABLE = "residuals_abs"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [18]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
y_pred = data_results[variable].rename("y_pred")
y_true = data_results[f"{variable}_true"].rename("y_true")
idata = np.abs(y_true - y_pred).mean(dim=[covariate]).load().rename(PLOT_VARIABLE)

In [19]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable_redfeten(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (0.0, 10, 7.5)
)

2024-12-17 11:08:32.793 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:321 - Starting Static Figures...
2024-12-17 11:08:32.794 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:328 - Plotting RESIDUALS_ABS Density...
2024-12-17 11:08:33.007 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:353 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/residuals_abs/posterior_predictive/residuals_abs_density_groups.pdf
2024-12-17 11:08:33.007 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:356 - Plotting Map...
2024-12-17 11:08:33.008 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable_redfeten:365 - Plotting Map of Spain...
2024-12-17 11:08:33.310 | DEBUG    | st_evt

## Model Errors - All

### Negative Log-Likelihood - All

In [20]:
subfigures_path = figures_path.joinpath("nll")
subfigures_path.mkdir(parents=True, exist_ok=True)

#### Posterior

In [21]:
variable = "t2max"
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="log_likelihood", num_samples=4000).median(dim=["sample"]).load()
isub_post = idata.sortby("gmst")[variable].rename("nll").sum(dim=covariate)

#### Posterior - Predictive

In [22]:

az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=4000).median(dim=["sample"]).load()
isub_postpred = idata.sortby("gmst")["nll"].sum(dim=covariate)

In [23]:
# labels
cbar_label = "Negative Log Predictive Density\n"
cbar_label += r"$\boldsymbol{L}(\mathbf{y};\boldsymbol{\theta},\boldsymbol{\phi},\mathcal{D})$"

#### Density

In [24]:
returns = []

returns.append({
    "period": "Red Feten (Posterior)",
    "color": "gray",
    "values":  isub_post.values.ravel(),
    "linestyle": "-",
    "values_units": "", # "[mm/day]",
})

returns.append({
    "period": "Red Feten (Posterior Predictive)",
    "color": "tab:blue",
    "values":  isub_postpred.where(isub_postpred.red_feten_mask == 1, drop=True).values.ravel(),
    "linestyle": "-",
    "values_units": "", # "[mm/day]",
})
returns.append({
    "period": "Not Red Feten (Posterior Predictive)",
    "color": "tab:red",
    "linestyle": "--",
    "values":  isub_postpred.where(isub_postpred.red_feten_mask == 0, drop=True).values.ravel(),
    "values_units": "", # "[mm/day]",
})

In [1]:
fig, ax = plot_density_multiple(returns, log_bins=False)
fig.set_size_inches(6, 5)
# fig.set_size_inches(8, 4.5)
ax.set(
    xlabel=cbar_label,
)

formatter = ScalarFormatter()
formatter.set_scientific(False)
ax.xaxis.set_major_formatter(formatter)
plt.legend(fontsize=10, loc="upper left")
plt.tight_layout()
fig.savefig(subfigures_path.joinpath("density_all.png"))
plt.show()

NameError: name 'plot_density_multiple' is not defined

## Global Parameters

### Concentration

In [9]:
PLOT_VARIABLE = "concentration"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [11]:
data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = data_results[PLOT_VARIABLE]
data_results

<xarray.DataArray 'concentration' (sample: 5000)> Size: 40kB
array([-0.28332204, -0.28695027, -0.28144797, ..., -0.28464127,
       -0.27646009, -0.28501931])
Coordinates:
  * sample   (sample) object 40kB MultiIndex
  * chain    (sample) int64 40kB 15 7 11 11 10 10 7 0 12 ... 12 14 0 6 15 7 10 4
  * draw     (sample) int64 40kB 131 265 965 625 460 737 ... 520 879 906 402 458

In [28]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
)

2024-12-17 11:08:59.618 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:08:59.618 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting CONCENTRATION Density...
2024-12-17 11:08:59.802 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/concentration/posterior/concentration_density.pdf
2024-12-17 11:08:59.802 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:00.162 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trai

### Kernel Parameters

#### Location Intercept - Kernel Variance

In [29]:
PLOT_VARIABLE = "location_intercept_kernel_variance"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [30]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [31]:
subfigures_path = Path(figures_path).joinpath(f"location_intercept/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
)

2024-12-17 11:09:00.698 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:00.699 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_INTERCEPT_KERNEL_VARIANCE Density...
2024-12-17 11:09:01.088 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_intercept/posterior/model_params/location_intercept_kernel_variance_density.pdf
2024-12-17 11:09:01.089 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:01.445 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/wa

#### Location Intercept - Kernel Length Scale

In [32]:
PLOT_VARIABLE = "location_intercept_kernel_scale"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [33]:
idata = az_ds.posterior[PLOT_VARIABLE]

In [34]:
subfigures_path = Path(figures_path).joinpath(f"location_intercept/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata.sel(spherical="lon").rename(f"{PLOT_VARIABLE}_x"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_x"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="lat").rename(f"{PLOT_VARIABLE}_y"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_y"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="alt").rename(f"{PLOT_VARIABLE}_z"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_z"],
)

2024-12-17 11:09:01.848 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:01.848 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_INTERCEPT_KERNEL_SCALE_X Density...
2024-12-17 11:09:02.029 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_intercept/posterior/model_params/location_intercept_kernel_scale_x_density.pdf
2024-12-17 11:09:02.029 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:02.338 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walk

#### Location Slope - Kernel Variance

In [35]:
PLOT_VARIABLE = "location_slope_kernel_variance"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [36]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [37]:
subfigures_path = Path(figures_path).joinpath(f"location_slope/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
)

2024-12-17 11:09:04.579 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:04.579 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_SLOPE_KERNEL_VARIANCE Density...
2024-12-17 11:09:04.746 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_slope/posterior/model_params/location_slope_kernel_variance_density.pdf
2024-12-17 11:09:04.747 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:05.038 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/ae

#### Location Slope - Kernel Length Scale

In [38]:
PLOT_VARIABLE = "location_slope_kernel_scale"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [39]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [40]:
subfigures_path = Path(figures_path).joinpath(f"location_slope/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata.sel(spherical="lon").rename(f"{PLOT_VARIABLE}_x"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_x"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="lat").rename(f"{PLOT_VARIABLE}_y"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_y"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="alt").rename(f"{PLOT_VARIABLE}_z"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_z"],
)

2024-12-17 11:09:05.470 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:05.471 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_SLOPE_KERNEL_SCALE_X Density...
2024-12-17 11:09:05.847 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_slope/posterior/model_params/location_slope_kernel_scale_x_density.pdf
2024-12-17 11:09:05.847 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:06.215 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aeme

#### Scale - kernel Variance

In [41]:
PLOT_VARIABLE = "scale_kernel_variance"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [42]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [43]:
subfigures_path = Path(figures_path).joinpath(f"scale/posterior/model_params/")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
)

2024-12-17 11:09:08.212 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:08.213 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting SCALE_KERNEL_VARIANCE Density...
2024-12-17 11:09:08.386 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/scale/posterior/model_params/scale_kernel_variance_density.pdf
2024-12-17 11:09:08.387 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:08.730 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_

#### Scale - Kernel Length Scale

In [44]:
PLOT_VARIABLE = "scale_kernel_scale"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [45]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [46]:
subfigures_path = Path(figures_path).joinpath(f"scale/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata.sel(spherical="lon").rename(f"{PLOT_VARIABLE}_x"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_x"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="lat").rename(f"{PLOT_VARIABLE}_y"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_y"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="alt").rename(f"{PLOT_VARIABLE}_z"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_z"],
)

2024-12-17 11:09:09.173 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:09.173 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting SCALE_KERNEL_SCALE_X Density...
2024-12-17 11:09:09.356 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/scale/posterior/model_params/scale_kernel_scale_x_density.pdf
2024-12-17 11:09:09.357 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:09.711 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp

### Mean Parameters

#### Location Intercept - Mean Intercept

In [47]:
PLOT_VARIABLE = "location_intercept_mean_intercept"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [49]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [50]:
subfigures_path = Path(figures_path).joinpath(f"location_intercept/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
)

2024-12-17 11:09:28.922 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:28.923 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_INTERCEPT_MEAN_INTERCEPT Density...
2024-12-17 11:09:29.095 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_intercept/posterior/model_params/location_intercept_mean_intercept_density.pdf
2024-12-17 11:09:29.096 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:29.389 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walk

#### Location Intercept - Mean Slope

In [51]:
PLOT_VARIABLE = "location_intercept_mean_slope"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [52]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [53]:
subfigures_path = Path(figures_path).joinpath(f"location_intercept/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata.sel(spherical="lon").rename(f"{PLOT_VARIABLE}_x"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_x"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="lat").rename(f"{PLOT_VARIABLE}_y"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_y"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="alt").rename(f"{PLOT_VARIABLE}_z"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_z"],
)

2024-12-17 11:09:30.862 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:30.862 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_INTERCEPT_MEAN_SLOPE_X Density...
2024-12-17 11:09:31.044 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_intercept/posterior/model_params/location_intercept_mean_slope_x_density.pdf
2024-12-17 11:09:31.044 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:31.373 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthro

#### Location Slope - Mean Intercept

In [54]:
PLOT_VARIABLE = "location_slope_mean_intercept"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [55]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [56]:
subfigures_path = Path(figures_path).joinpath(f"location_slope/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
)

2024-12-17 11:09:33.441 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:33.441 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_SLOPE_MEAN_INTERCEPT Density...
2024-12-17 11:09:33.621 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_slope/posterior/model_params/location_slope_mean_intercept_density.pdf
2024-12-17 11:09:33.621 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:33.905 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aeme

#### Location Slope - Mean Slope

In [57]:
PLOT_VARIABLE = "location_slope_mean_slope"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [58]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [59]:
subfigures_path = Path(figures_path).joinpath(f"location_slope/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata.sel(spherical="lon").rename(f"{PLOT_VARIABLE}_x"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_x"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="lat").rename(f"{PLOT_VARIABLE}_y"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_y"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="alt").rename(f"{PLOT_VARIABLE}_z"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_z"],
)

2024-12-17 11:09:35.900 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:35.901 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting LOCATION_SLOPE_MEAN_SLOPE_X Density...
2024-12-17 11:09:36.076 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_slope/posterior/model_params/location_slope_mean_slope_x_density.pdf
2024-12-17 11:09:36.076 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:36.639 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2

#### Scale - Mean Intercept

In [60]:
PLOT_VARIABLE = "scale_mean_intercept"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [61]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [62]:
subfigures_path = Path(figures_path).joinpath(f"scale/posterior/model_params/")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
)

2024-12-17 11:09:38.704 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:38.705 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting SCALE_MEAN_INTERCEPT Density...
2024-12-17 11:09:38.901 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/scale/posterior/model_params/scale_mean_intercept_density.pdf
2024-12-17 11:09:38.902 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:39.176 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp

#### Scale - Mean Slope

In [63]:
PLOT_VARIABLE = "scale_mean_slope"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [64]:
# data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
# data_results = data_results.sortby("gmst")
idata = az_ds.posterior[PLOT_VARIABLE]

In [65]:
subfigures_path = Path(figures_path).joinpath(f"scale/posterior/model_params")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_global_variable(
    idata.sel(spherical="lon").rename(f"{PLOT_VARIABLE}_x"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_x"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="lat").rename(f"{PLOT_VARIABLE}_y"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_y"],
)
utils_region.plot_static_global_variable(
    idata.sel(spherical="alt").rename(f"{PLOT_VARIABLE}_z"),
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_z"],
)

2024-12-17 11:09:39.916 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:182 - Starting Static Figures...
2024-12-17 11:09:39.916 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:189 - Plotting SCALE_MEAN_SLOPE_X Density...
2024-12-17 11:09:40.100 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:202 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/scale/posterior/model_params/scale_mean_slope_x_density.pdf
2024-12-17 11:09:40.101 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:207 - Plotting Parameter Traces...
2024-12-17 11:09:40.416 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_global_variable:217 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcm

## Static Spatial Variables

### Scale - Posterior

In [66]:
PLOT_VARIABLE = "scale"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [69]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior", num_samples=num_samples).median(dim=["sample"]).load()
idata = idata.sortby("gmst")
idata = idata[PLOT_VARIABLE]

In [70]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (0.5, 10, 3.0)
)

2024-12-17 11:10:16.091 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:262 - Starting Static Figures...
2024-12-17 11:10:16.092 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:269 - Plotting SCALE Density...
2024-12-17 11:10:16.276 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:282 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/scale/posterior/scale_density.pdf
2024-12-17 11:10:16.277 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:285 - Plotting Map...
2024-12-17 11:10:16.277 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:294 - Plotting Map of Spain...
2024-12-17 11:10:16.555 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:304 - Saved Figure

### Scale - Posterior Predictive

In [71]:
PLOT_VARIABLE = "scale"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [72]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
idata = idata.sortby("gmst")
idata = idata[PLOT_VARIABLE]

In [73]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (0.5, 10, 3.0)
)

2024-12-17 11:11:04.938 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:262 - Starting Static Figures...
2024-12-17 11:11:04.939 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:269 - Plotting SCALE Density...
2024-12-17 11:11:05.135 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:282 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/scale/posterior_predictive/scale_density.pdf
2024-12-17 11:11:05.136 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:285 - Plotting Map...
2024-12-17 11:11:05.136 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:294 - Plotting Map of Spain...
2024-12-17 11:11:05.427 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:304 - S

### Location Intercept - Posterior

In [74]:
PLOT_VARIABLE = "location_intercept"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [75]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior", num_samples=num_samples).median(dim=["sample"]).load()
idata = idata.sortby("gmst")
idata = idata[PLOT_VARIABLE]

In [76]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (20.0, 10, 45.0)
)

2024-12-17 11:12:13.137 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:262 - Starting Static Figures...
2024-12-17 11:12:13.137 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:269 - Plotting LOCATION_INTERCEPT Density...
2024-12-17 11:12:13.319 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:282 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_intercept/posterior/location_intercept_density.pdf
2024-12-17 11:12:13.320 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:285 - Plotting Map...
2024-12-17 11:12:13.320 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:294 - Plotting Map of Spain...
2024-12-17 11:12:13.598 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_sta

### Location Intercept - Posterior Predictive

In [82]:
PLOT_VARIABLE = "location_intercept"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [83]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).mean(dim=["sample"]).load()
idata = idata.sortby("gmst")
idata = idata[PLOT_VARIABLE]

In [84]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (20.0, 10, 45.0)
)

2024-12-17 11:14:25.967 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:262 - Starting Static Figures...
2024-12-17 11:14:25.968 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:269 - Plotting LOCATION_INTERCEPT Density...
2024-12-17 11:14:26.159 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:282 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_intercept/posterior_predictive/location_intercept_density.pdf
2024-12-17 11:14:26.159 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:285 - Plotting Map...
2024-12-17 11:14:26.160 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:294 - Plotting Map of Spain...
2024-12-17 11:14:26.457 | DEBUG    | st_evt._src.modules.models.aemet.utils_regi

### Location Slope - Posterior

In [85]:
PLOT_VARIABLE = "location_slope"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [86]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior", num_samples=num_samples).median(dim=["sample"]).load()
idata = idata.sortby("gmst")
idata = idata[PLOT_VARIABLE]

In [87]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (0.5, 8, 2.5)
)

2024-12-17 11:14:47.194 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:262 - Starting Static Figures...
2024-12-17 11:14:47.195 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:269 - Plotting LOCATION_SLOPE Density...
2024-12-17 11:14:47.377 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:282 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_slope/posterior/location_slope_density.pdf
2024-12-17 11:14:47.378 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:285 - Plotting Map...
2024-12-17 11:14:47.378 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:294 - Plotting Map of Spain...
2024-12-17 11:14:47.680 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_

### Location Slope - Posterior Predictive

In [88]:
PLOT_VARIABLE = "location_slope"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [89]:
az_ds = az.from_zarr(store=str(results_data_path))
idata = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
idata = idata.sortby("gmst")
idata = idata[PLOT_VARIABLE]

In [90]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_static_spatial_variable(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (0.5, 8, 2.5)
)

2024-12-17 11:15:31.799 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:262 - Starting Static Figures...
2024-12-17 11:15:31.800 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:269 - Plotting LOCATION_SLOPE Density...
2024-12-17 11:15:31.988 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:282 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_slope/posterior_predictive/location_slope_density.pdf
2024-12-17 11:15:31.988 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:285 - Plotting Map...
2024-12-17 11:15:31.989 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_static_spatial_variable:294 - Plotting Map of Spain...
2024-12-17 11:15:32.284 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_stat

## Dynamic Spatial Variables

### Locaiton - Posterior

In [91]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [92]:
data_results = az.extract(az_ds, group="posterior", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [93]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (20, 10, 45),
    covariate = covariate,
    units = "[°C]"
)

2024-12-17 11:16:06.936 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:404 - Plotting LOCATION Density...
2024-12-17 11:16:07.149 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:428 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location/posterior/location_density_groups.png
2024-12-17 11:16:07.149 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:431 - Plotting Map...
2024-12-17 11:16:07.150 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:441 - Plotting Map of Spain...
2024-12-17 11:16:07.429 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:447 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_non

### Locaiton - Posterior Predictive

In [100]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 10_000

In [101]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [102]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (20, 10, 45),
    covariate = covariate,
    units = "[°C]"
)

2024-12-17 11:17:56.299 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:404 - Plotting LOCATION Density...
2024-12-17 11:17:56.517 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:428 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location/posterior_predictive/location_density_groups.png
2024-12-17 11:17:56.518 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:431 - Plotting Map...
2024-12-17 11:17:56.519 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:441 - Plotting Map of Spain...
2024-12-17 11:17:56.808 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:447 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t

### Locaiton - Predictions

In [97]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [98]:
data_results = az.extract(az_ds, group="predictions", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
idata = data_results[PLOT_VARIABLE]
idata

<xarray.DataArray 'location' (gmst: 100, station_id: 699)> Size: 559kB
array([[32.48081766, 32.90908062, 33.23638697, ..., 51.99050508,
        28.8167991 , 24.77065629],
       [32.53227108, 32.95760955, 33.27885734, ..., 52.04596758,
        28.85624955, 24.80910901],
       [32.58060412, 33.00714962, 33.32526173, ..., 52.07942104,
        28.89856502, 24.84468725],
       ...,
       [37.26652632, 37.67176332, 37.71476646, ..., 56.67123825,
        32.69642997, 28.22013363],
       [37.31573222, 37.72096319, 37.76302462, ..., 56.73595148,
        32.74266901, 28.25798037],
       [37.36477137, 37.76966592, 37.81005064, ..., 56.79266778,
        32.7817723 , 28.29562328]])
Coordinates:
    alt             (station_id) float32 3kB 32.0 118.0 487.0 ... 620.0 10.0
  * gmst            (gmst) float64 800B 0.0 0.02525 0.05051 ... 2.449 2.475 2.5
    lat             (station_id) float32 3kB 40.96 41.15 41.38 ... 27.98 28.13
    lon             (station_id) float32 3kB 0.87 1.108 1.081 ... -15.5 -15.42
    red_feten_mask  (station_id) uint8 699B 1 1 0 0 1 0 0 0 ... 0 1 0 0 0 0 0 1
  * station_id      (station_id) <U5 14kB '0002I' '0016B' ... 'C646O' 'C659T'
    station_name    (station_id) <U50 140kB 'vandellòs' ... 'las palmas de g....

In [91]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/predictions")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_gmst_pred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (20, 10, 45),
    covariate = covariate
)

2024-12-17 11:04:49.523 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:547 - Plotting LOCATION Density...
2024-12-17 11:04:50.107 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:578 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location/predictions/location_density_groups.png
2024-12-17 11:04:50.108 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:581 - Plotting Map...
2024-12-17 11:04:50.109 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:591 - Plotting Map of Spain...
2024-12-17 11:04:50.394 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:597 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max

### Locaiton Difference - Posterior

In [103]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 10_000

In [104]:
data_results = az.extract(az_ds, group="posterior", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [105]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}_difference/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_diff_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_difference"],
    cmap="Reds",
    bounds = (1.0, 10, 2.5),
    covariate = covariate,
    units = "[°C]"
)

2024-12-17 11:18:31.764 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:484 - Plotting LOCATION_DIFFERENCE Density...
2024-12-17 11:18:31.929 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:497 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_difference/posterior/location_difference_density.png
2024-12-17 11:18:31.930 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:500 - Plotting Map...
2024-12-17 11:18:31.930 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:509 - Plotting Map of Spain...
2024-12-17 11:18:32.216 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:519 - Saved Figure: 
/home/juanjohn/pool_data/

### Locaiton Difference - Posterior Predictive

In [106]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 10_000

In [107]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [108]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}_difference/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_diff_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_difference"],
    cmap="Reds",
    bounds = (1.0, 10, 2.5),
    covariate = covariate,
    units = "[°C]"
)

2024-12-17 11:19:13.216 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:484 - Plotting LOCATION_DIFFERENCE Density...
2024-12-17 11:19:13.394 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:497 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_difference/posterior_predictive/location_difference_density.png
2024-12-17 11:19:13.394 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:500 - Plotting Map...
2024-12-17 11:19:13.395 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:509 - Plotting Map of Spain...
2024-12-17 11:19:13.693 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_postpred:519 - Saved Figure: 
/home/juanjohn

### Locaiton Difference - Predictions

In [109]:
PLOT_VARIABLE = "location"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 10_000

In [110]:
data_results = az.extract(az_ds, group="predictions", num_samples=num_samples).median(dim=["sample"]).load()
data_results = data_results.sortby("gmst")
idata = data_results[PLOT_VARIABLE]

In [111]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}_difference/predictions")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_diff_gmst_pred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_difference"],
    cmap="Reds",
    bounds = (1.0, 10, 2.5),
    covariate = covariate,
    periods = [0.0, 1.3, 2.5]
)

2024-12-17 11:19:40.233 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_gmst_pred:638 - Plotting LOCATION Density...
2024-12-17 11:19:40.460 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_gmst_pred:662 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/location_difference/predictions/location_density_groups.png
2024-12-17 11:19:40.460 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_gmst_pred:665 - Plotting Map...
2024-12-17 11:19:40.461 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_gmst_pred:674 - Plotting Map of Spain...
2024-12-17 11:19:40.759 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_diff_gmst_pred:680 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo

## Dynamic Spatial Variables - Post Processing

### 100-Year Return Period - Posterior

In [19]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 2_000

In [20]:
data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_station.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()

In [21]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (25, 10, 50),
    covariate = covariate,
    units = "[°C]"
)

2024-12-17 11:27:46.104 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:404 - Plotting RETURN_LEVEL_100 Density...
2024-12-17 11:27:46.464 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:428 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/return_level_100/posterior/return_level_100_density_groups.png
2024-12-17 11:27:46.464 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:431 - Plotting Map...
2024-12-17 11:27:46.466 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:441 - Plotting Map of Spain...
2024-12-17 11:27:47.279 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:447 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkth

### 100-Year Return Level - Posterior Predictive

In [22]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 2_000

In [23]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_station.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()

In [24]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (25, 10, 50),
    covariate = covariate,
    units = "[°C]"
)

2024-12-17 11:29:16.146 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:404 - Plotting RETURN_LEVEL_100 Density...
2024-12-17 11:29:16.371 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:428 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/return_level_100/posterior_predictive/return_level_100_density_groups.png
2024-12-17 11:29:16.371 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:431 - Plotting Map...
2024-12-17 11:29:16.372 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:441 - Plotting Map of Spain...
2024-12-17 11:29:16.663 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_postpred:447 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experim

### 100-Year Return Level - Predictions

In [25]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 1_000

In [26]:
data_results = az.extract(az_ds, group="predictions", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_station.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()

In [27]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}/predictions")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_gmst_pred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[PLOT_VARIABLE],
    cmap="Reds",
    bounds = (25, 10, 50),
    covariate = covariate
)

2024-12-17 11:31:47.000 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:547 - Plotting RETURN_LEVEL_100 Density...
2024-12-17 11:31:47.972 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:578 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments/walkthrough/aemet/t2max/az_nonstationary_gp_mcmc_redfeten_trainnoise/results/figures/spain/return_level_100/predictions/return_level_100_density_groups.png
2024-12-17 11:31:47.973 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:581 - Plotting Map...
2024-12-17 11:31:47.974 | INFO     | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:591 - Plotting Map of Spain...
2024-12-17 11:31:48.381 | DEBUG    | st_evt._src.modules.models.aemet.utils_region:plot_dynamic_spatial_variable_gmst_pred:597 - Saved Figure: 
/home/juanjohn/pool_data/dynev4eo/experiments

### 100-Year Return Level Difference - Posterior

In [ ]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 10_000

In [ ]:
data_results = az.extract(az_ds, group="posterior", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_station.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}_difference/posterior")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_diff_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_difference"],
    cmap="Reds",
    bounds = (1.0, 10, 2.5),
    covariate = covariate,
    units = "[°C]"
)

### 100-Year Return Level Difference - Posterior Predictive

In [ ]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 5_000

In [ ]:
data_results = az.extract(az_ds, group="posterior_predictive", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_station.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()
idata

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}_difference/posterior_predictive")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_diff_postpred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_difference"],
    cmap="Reds",
    bounds = (1.0, 10, 2.5),
    covariate = covariate,
    units = "[°C]"
)

### 100-Year Return Level Difference - Predictions

In [ ]:
PLOT_VARIABLE = "return_level_100"
VARIABLE = "t2max"
COVARIATE = "gmst"
num_samples = 10_000

In [ ]:
data_results = az.extract(az_ds, group="predictions", num_samples=num_samples)
data_results = data_results.sortby("gmst")
data_results = utils_station.calculate_ds_return_periods(data_results, ["sample"])
idata = data_results[PLOT_VARIABLE].median(dim=["sample"]).load()
idata

In [ ]:
subfigures_path = Path(figures_path).joinpath(f"{PLOT_VARIABLE}_difference/predictions")
subfigures_path.mkdir(parents=True, exist_ok=True)

utils_region.plot_dynamic_spatial_variable_diff_gmst_pred(
    idata,
    figures_path=subfigures_path,
    cbar_label=utils_region.VARIABLE_LABELS[f"{PLOT_VARIABLE}_difference"],
    cmap="Reds",
    bounds = (1.0, 10, 2.5),
    covariate = covariate,
    periods = [0.0, 1.3, 2.5]
)